# Tutorial with Toy Examples for Joint Cutting in `qsimh`

This file runs a few toy examples and shows how joint cutting can be employed. Furthermore utility functions are used to read out the output files.

In [6]:
import sys
sys.path.append("../")
import readout


One can compare the HSF runtimes both for the traditional cutting and the joint cut. First, construct an example circuit for the traditional cutting

In [7]:
!cat example_circuit_q20_noblock

20
0 h 0
0 h 1
0 h 2
0 h 3
0 h 4
0 h 5
0 h 6
0 h 7
0 h 8
0 h 9
0 h 10
0 h 11
0 h 12
0 h 13
0 h 14
0 h 15
0 h 16
0 h 17
0 h 18
0 h 19
1 is 9 10
2 rzz 8 11 0.24
3 rzz 9 11 1.52
4 cz 8 10

and adding brackets for a joint cut of a block of gates as:

In [8]:
!cat example_circuit_q20_block

20
0 h 0
0 h 1
0 h 2
0 h 3
0 h 4
0 h 5
0 h 6
0 h 7
0 h 8
0 h 9
0 h 10
0 h 11
0 h 12
0 h 13
0 h 14
0 h 15
0 h 16
0 h 17
0 h 18
0 h 19
[1 is 9 10
2 rzz 8 11 0.24
3 rzz 9 11 1.52
4 cz 8 10]

Now run the two different HSF simulations. First the traditional cut, with parameters specified as follows:

In [9]:
%%bash
#!/bin/bash

d_flag=100
q=20
t=8
r_block=4
cutloc=9
k_values="0,1,2,3,4,5,6,7,8,9"
reps=5

circname_noblock="./example_circuit_q20_noblock"

for (( i=0; i<reps; i++)); do
    #run example with cut but no block
    filename0="./times_noblock_q20_rep${i}.tlog"
    amps0="./amplitudes_noblock_q20.log"
    echo "Starting Cut (no block)..."
    ../../apps/qsimh_amplitudes.x -c ${circname_noblock} -d ${d_flag} -k ${k_values} -t ${t} -w 0 -p 0 -r ${r_block}  -i ../../circuits/bitstrings_q${q} -o "$amps0" -v 1 > "$filename0"
    echo "Completed: $filename0"
done

Starting Cut (no block)...
Completed: ./times_noblock_q20_rep0.tlog
Starting Cut (no block)...
Completed: ./times_noblock_q20_rep1.tlog
Starting Cut (no block)...
Completed: ./times_noblock_q20_rep2.tlog
Starting Cut (no block)...
Completed: ./times_noblock_q20_rep3.tlog
Starting Cut (no block)...
Completed: ./times_noblock_q20_rep4.tlog


Then, we do the same for the joint cut:

In [10]:
%%bash
#!/bin/bash

d_flag=100
q=20
t=8
r_block=1
cutloc=9
k_values="0,1,2,3,4,5,6,7,8,9"
reps=5

circname_block="./example_circuit_q20_block"

for (( i=0; i<reps; i++)); do
    #run example with cut but no block
    filename0="./times_block_q20_rep${i}.tlog"
    amps0="./amplitudes_block_q20.log"
    echo "Starting Cut (block)..."
    ../../apps/qsimh_amplitudes.x -c ${circname_block} -d 100 -k ${k_values} -t ${t} -w 0 -p 0 -r ${r_block}  -i ../../circuits/bitstrings_q${q} -o "$amps0" -v 1 > "$filename0"
    echo "Completed: $filename0"
done

Starting Cut (block)...
Completed: ./times_block_q20_rep0.tlog
Starting Cut (block)...
Completed: ./times_block_q20_rep1.tlog
Starting Cut (block)...
Completed: ./times_block_q20_rep2.tlog
Starting Cut (block)...
Completed: ./times_block_q20_rep3.tlog
Starting Cut (block)...
Completed: ./times_block_q20_rep4.tlog


For reference, one can also run the `qsim` simulation without any cutting (note that qsim could not handle the input circuit files with brackets):

In [11]:
%%bash
#!/bin/bash

d_flag=100
q=20
t=8
cutloc=9
reps=5

circname_noblock="./example_circuit_q20_noblock"

for (( i=0; i<reps; i++)); do
    #run example with cut but no block
    filename0="./times_nocut_q20_rep${i}.tlog"
    amps0="./amplitudes_nocut_q20.log"
    echo "Starting No cut..."
    ../../apps/qsim_amplitudes.x -c ${circname_noblock} -d ${d_flag} -i ../../circuits/bitstrings_q${q} -o "$amps0" -v 2 -t ${t} > "$filename0"
    echo "Completed: $filename0"
done

Starting No cut...
Completed: ./times_nocut_q20_rep0.tlog
Starting No cut...
Completed: ./times_nocut_q20_rep1.tlog
Starting No cut...
Completed: ./times_nocut_q20_rep2.tlog
Starting No cut...
Completed: ./times_nocut_q20_rep3.tlog
Starting No cut...
Completed: ./times_nocut_q20_rep4.tlog


One can readout the output files and compare the runtimes as well as checking the amplitudes:

In [19]:

#amplitude checking
i=0
num_amps=1000000
t=8
amps_nocut = "./amplitudes_nocut_q20.log"
amps_block = "./amplitudes_block_q20.log"
amps_noblock = "./amplitudes_noblock_q20.log"

readout.compare_files(amps_block, amps_nocut)

#print times
reps=5
suffix = ""
path_nocut = "./times_nocut_q20"
path_block = "./times_block_q20"
path_noblock = "./times_noblock_q20"
readout.print_diffs(reps, path_nocut, path_block, path_noblock, suffix)


All fine within the given tolerance 1e-07
--------Full times---------
No cut: Mean =1.462634, std=0.07408775569552638
Block: Mean =0.05619302, std=0.009142115711015695
No Block: Mean =0.14016802, std=0.045074792069643536
--------Sim times---------
No cut: Mean =1.4560380000000002, std=0.07220976122381237
Block: Mean =0.051568159999999995, std=0.00946799451554552
No Block: Mean =0.13703904, std=0.04514148058278993
-----Paths-----
Block: 8
No Block: 32
--------Ratios--------
S/J = 26.02874876630585
T/J = 2.4944026856004533


This tells us that the amplitudes are fine within the tolerance. It also compares the runtimes of all the different runs, also considering the number of paths for each the traditional and joint cutting as well as the ratios between the Schrödinger and Joint Cutting (S/J) as well as the traditional divicded by the joint cutting (T/J). In this small example, the joint cutting is already beneficial. 